In [ ]:
day_obs = -1
sim_filename = '/Users/lynnej/opsim/fbs_3.2/baseline_v3.2_10yrs.db'
version = 3
sim_filename = ''
version = 5
calculate = True

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import healpy as hp
import sqlite3
from IPython.display import display, Markdown

from astropy.time import Time, TimeDelta
import astropy.units as u

from rubin_scheduler.scheduler.utils import get_current_footprint
import rubin_sim.maf as maf
from rubin_sim.data import get_baseline

# Survey Strategy Key Metrics

Use the baseline survey (eventually real visits + new sim) to calculate 
* Sky Coverage (LSR-REQ-0098)
* Fast Revisit Area (LSR-REQ-0098)
* Proper Motion (LSR-REQ-0099)
* Parallax Uncertainty (LSR-REQ-0099)
* Survey Time Allocation (LSR-REQ-0075) 

In [ ]:
if sim_filename is not None and len(sim_filename) > 0:
    opsdb = sim_filename
else:
    opsdb = get_baseline()
    
run_name = os.path.split(opsdb)[-1].replace(".db", "")

print(f"Evaluating {run_name}")

In [ ]:
nside = 64
pixarea = hp.nside2pixarea(nside, degrees=True)
wfd_labels = ["lowdust", "euclid_overlap", "virgo", "bulgy", "LMC_SMC"]
footprints, labels = get_current_footprint(nside=nside)
wfdpix = np.where(np.isin(labels, wfd_labels), 1, 0)


colmap = maf.col_map_dict("fbs")
if version < 4:
    colmap = maf.col_map_dict("fbs_sim")
    colmap['scheduler_note'] = 'note'

slicer = maf.HealpixSlicer(nside=nside, lon_col=colmap['ra'], lat_col=colmap['dec'], use_cache=True, verbose=False)
wfd_slicer = maf.HealpixSubsetSlicer(nside=nside, hpid=np.where(wfdpix == 1)[0], lon_col=colmap['ra'], lat_col=colmap['dec'], use_cache=True, verbose=False)
s = wfd_slicer

In [ ]:
# Calculate over whole sky; we can mask later. 

fO_bundles = maf.fOBatch(colmap=colmap,
                         slicer=slicer,
                        run_name=run_name,
                        benchmark_area=18000,
                        benchmark_n_visits=825,
                        min_n_visits=750,)
revisit_bundles = maf.rapidRevisitBatch(colmap=colmap, 
                                        slicer=s,
                                        run_name=run_name)
astrometry_bundles = maf.astrometryBatch(colmap=colmap, 
                                         slicer=s,
                                        run_name=run_name)
# Let's drop some of the astrometry bundles 
keep_keys = [k for k in astrometry_bundles if "Normalized" not in k and "Coverage" not in k and "degeneracy" not in k]
astrometry_bundles = dict([(k, astrometry_bundles[k]) for k in keep_keys])

slew_bundles = maf.slewBasics(colmap=colmap, 
                              run_name=run_name)

# Nvisits for WFD and for all sky
bdict = {}
bdict.update(
    maf.nvisitsPerSubset(
        colmap,
        run_name,
        constraint="visitExposureTime > 19",
        footprintConstraint=wfdpix,
        extraInfoLabel="WFD",
    )
)
dd_constraint = f"{colmap['scheduler_note']} like '%DD%'"
bdict.update(
    maf.nvisitsPerSubset(
        colmap,
        run_name,
        constraint=dd_constraint,
        footprintConstraint=None,
        extraInfoLabel="DDF",
    )
)
bdict.update(
    maf.nvisitsPerSubset(
        colmap,
        run_name,
        constraint=None,
        footprintConstraint=None,
        extraInfoLabel="All visits",
    )
)


all_bundles = {}
all_bundles.update(fO_bundles)
all_bundles.update(revisit_bundles)
all_bundles.update(astrometry_bundles)
all_bundles.update(slew_bundles)
all_bundles.update(bdict)

In [ ]:
if calculate:
    g = maf.MetricBundleGroup(all_bundles, opsdb, save_early=False, verbose=False)
    g.run_all()

In [ ]:
astrometry_bundles_ideal = maf.astrometryBatch(colmap=colmap, 
                                         slicer=s,
                                         run_name=run_name)

seeing_srd = {'u': 0.77, 'g': 0.73, 'r': 0.70, 'i' : 0.67, 'z': 0.65, 'y': 0.63}
m5_srd = {'u': 23.90, 'g': 25.0, 'r': 24.70, 'i': 24.0, 'z': 23.3, 'y': 22.1}

if calculate:
    # Hack visits to all have fiducial seeing and depth
    conn = sqlite3.connect(opsdb)
    query = 'select * from observations'
    dd = pd.read_sql(query, conn)
    dd = dd.to_records()
    for f in 'ugrizy':
        good = np.where(dd['filter'] == f)
        dd['seeingFwhmGeom'][good] = seeing_srd[f]
        dd['fiveSigmaDepth'][good] = m5_srd[f]
    
    g = maf.MetricBundleGroup(astrometry_bundles_ideal, opsdb, save_early=False)
    ## all bands
    g.run_current(constraint='', sim_data=dd)
    ## y band
    kk = f"{run_name.replace('.', '_')}_Parallax_Uncert_@_21_3_y_band_visits_HEAL"
    sim_data = dd[np.where(dd["filter"] == "y")]
    g.run_current(constraint=astrometry_bundles_ideal[kk].constraint, sim_data=sim_data)

In [ ]:
if calculate:
    print("Metrics calculated.")
else:
    print("Skipping calculation.")

In [ ]:
display(Markdown("## Fraction of WFD visits ##"))

if calculate:
    # Fraction of visits counting in fO that are in WFD area
    fO = fO_bundles[f"{run_name.replace('.', '_')}_fO_All_visits_HEAL"]
    wfd_subset = fO.metric_values.filled(0)[np.where(wfdpix == 1)].sum() * pixarea
    all_visits = fO.metric_values.filled(0).sum() * pixarea
    display(Markdown(f"Visits within WFD region consist of {100*wfd_subset/all_visits : .1f} % of total visits."\
                     " This includes DDF visits, but not visits shorter than 20 seconds."))
    
    # Fraction of visits, counted differently, should be very similar
    k = f"{run_name.replace('.', '_')}_Nvisits_WFD_UNIS"
    wfd_visits = bdict[k].metric_values[0]
    k = f"{run_name.replace('.', '_')}_Nvisits_DDF_UNIS"
    ddf_visits = bdict[k].metric_values[0]
    k = f"{run_name.replace('.', '_')}_Nvisits_All_visits_UNIS"
    all_visits = bdict[k].metric_values[0]
    display(Markdown(f"Find {100 * wfd_visits / all_visits :.1f}% of visits in WFD, and {100* ddf_visits /all_visits :0.1f}% DDF, out of {all_visits :.0f} total."))

In [ ]:
display(Markdown("## Sky Coverage ##"))
if calculate:
    fO = fO_bundles[f"{run_name.replace('.', '_')}_fO_All_visits_HEAL"]
    nv_med = np.where(fO.summary_values['fONv']['name'] == 'MedianNvis')[0]
    nv_min = np.where(fO.summary_values['fONv']['name'] == 'MinNvis')[0]
    
    print(f"Asky is {fO.summary_values['fOArea_750'] :.1f}. Minimum requirement is 15000, Design requirement is 18000.")
    print(f"Nv1Sum is {fO.summary_values['fONv'][nv_med]['value'][0]}. Minimum requirement is 750. Design requirement is 825.")

In [ ]:
if calculate:
    # Plot cumulative area covered to a minimum number of visits
    ph = maf.PlotHandler(savefig=False)
    ph.set_metric_bundles([fO])
    
    plot_dict = {"color_min": 0, "color_max": 1100}
    _ = ph.plot(plot_func=maf.HealpixSkyMap(), plot_dicts=plot_dict)
    
    plot_dict = {"asky":18000, "n_visits": 750,}
    _ = ph.plot(plot_func=maf.FOPlot(), plot_dicts=plot_dict)

The dotted red and blue lines above indicate design AREA and minimum NVISIT SRD values. The solid blue line indicates the median number of visits over the top 18,000 sq degrees. The solid red line indicates the area covered to a minimum of 750 visits per pointing. The solid black line indicates the area covered to a minimum number of visits, as a function of minimum number of visits.

In [ ]:
display(Markdown("## Rapid Revisit ##"))
if calculate:
    revisit_2 = revisit_bundles[f"{run_name.replace('.', '_')}_RapidRevisits_All_visits_HEAL"]
    print(f"RVA1 - Area meeting rapid revisit requirement : {revisit_2.summary_values['Area (sq deg)'] :.1f}. Design requirement is 2000 sq degrees.")

In [ ]:
if calculate:
    plot_dict = {'color_min': 0, 'color_max': 1}
    revisit_2.set_plot_dict(plot_dict)
    revisit_2.plot_funcs = [maf.HealpixSkyMap()]
    revisit_2.plot()
    display(Markdown((revisit_2.display_dict['caption'])))

In [ ]:
display(Markdown("## Parallax Uncertainty ##"))

if calculate:
    display(Markdown("Median parallax uncertainty over the top 18k square degrees (approximately equivalent to the 'WFD') "\
        "under SRD seeing and depth conditions at r=24.0 is "))
    print("SIGpara = %.2f mas " % astrometry_bundles[f"{run_name.replace('.', '_')}_Parallax_Uncert_@_24_0_All_visits_HEAL"].summary_values["Median Parallax Uncert (18k)"])
    print("The design requirement is 3.0 mas, the minimum requirement is 6.0 mas.")
    
    print("")
    
    b = astrometry_bundles[f"{run_name.replace('.', '_')}_Parallax_Uncert_@_21_3_y_band_visits_HEAL"]
    display(Markdown("Median parallax uncertainty over the top 18k square degrees under SRD seeing and depth conditions, "\
                     "for sources with SNR=10 visible in y band only"))
    print("SIGparaRed %.2f mas" % b.summary_values["Median Parallax Uncert (18k)"])
    print("The design requirement is 6.0 mas, the minimum requirement is 10.0 mas.")
    
    display(Markdown("NOTE: design values use fiducial seeing and skybrightness for calculation, "\
            "while above calculation uses range of simulated values and thus is expected to be worse"))

In [ ]:
if calculate:
    display(Markdown("Evaluated with SRD fiducial seeing and M5 values "))
    print("SIGpara = %.2f mas " % astrometry_bundles_ideal[f"{run_name.replace('.', '_')}_Parallax_Uncert_@_24_0_All_visits_HEAL"].summary_values["Median Parallax Uncert (18k)"])
    print("The design requirement is 3.0 mas, the minimum requirement is 6.0 mas.")
    
    print("")
    
    b = astrometry_bundles_ideal[f"{run_name.replace('.', '_')}_Parallax_Uncert_@_21_3_y_band_visits_HEAL"]
    display(Markdown("Median parallax uncertainty over the top 18k square degrees under SRD seeing and depth conditions, "\
                     "for sources with SNR=10 visible in y band only"))
    print("SIGparaRed %.2f mas" % b.summary_values["Median Parallax Uncert (18k)"])
    print("The design requirement is 6.0 mas, the minimum requirement is 10.0 mas.")

In [ ]:
if calculate:
    print("Calculated values with model seeing and M5")
    b = astrometry_bundles[f"{run_name.replace('.', '_')}_Parallax_Uncert_@_24_0_All_visits_HEAL"]
    plot_dict = {"color_min" : 2, "color_max": 8, "bins": 150}
    b.set_plot_dict(plot_dict)
    _ = b.plot()

In [ ]:
display(Markdown("## Proper Motion ##"))

if calculate:
    display(Markdown(("Median proper motion uncertainty over the top 18k square degrees (approximately equivalent to the 'WFD') "\
            "under SRD seeing and depth conditions is at r=24.0 ")))
    print("SIGpm = %.2f mas " % astrometry_bundles[f"{run_name.replace('.', '_')}_Proper_Motion_Uncert_@_24_0_All_visits_HEAL"].summary_values["Median Proper Motion Uncert (18k)"])
    print("The design requirement is 1.0 mas, the minimum requirement is 2.0 mas.")
    display(Markdown("NOTE: design values use fiducial seeing and skybrightness for calculation, "\
            "while above calculation uses range of simulated values and thus is expected to be worse"))

In [ ]:
if calculate:
    display(Markdown(("Evaluated with SRD fiducial seeing and M5 values ")))
    print("SIGpm = %.2f mas " % astrometry_bundles_ideal[f"{run_name.replace('.', '_')}_Proper_Motion_Uncert_@_24_0_All_visits_HEAL"].summary_values["Median Proper Motion Uncert (18k)"])
    print("The design requirement is 1.0 mas, the minimum requirement is 2.0 mas.")

In [ ]:
if calculate:
    print("Calculated values with model seeing and M5")
    b = astrometry_bundles[f"{run_name.replace('.', '_')}_Proper_Motion_Uncert_@_24_0_All_visits_HEAL"]
    plot_dict = {"color_min" : 0.5, "color_max": 2, "bins": 150}
    b.set_plot_dict(plot_dict)
    _ = b.plot()

In [ ]:
display(Markdown("## Time between visits ##"))

if calculate:
    k = f"{run_name.replace('.', '_')}_Median_slewTime_All_visits_UNIS"
    b2 = slew_bundles[k]
    print(f"medianVisitInterval: Median slew time is {b2.metric_values[0]: .2f} seconds")
    print("Requirement is 5 seconds")
    
    print("")
    
    k = f"{run_name.replace('.', '_')}_Mean_slewTime_All_visits_UNIS"
    b1 = slew_bundles[k]
    print(f"aveVisitInterval: Mean slew time is {b1.metric_values[0]: .2f} seconds")
    print("Requirement is 10 seconds")

In [ ]:
if calculate:
    k = f"{run_name.replace('.', '_')}_Slew_Time_Histogram_All_visits_ONED"
    b = slew_bundles[k]
    b.set_plot_dict({'y_min': 10})
    label = f"Median slew time: {b2.metric_values[0]: .2f}s\n Mean slew time: {b1.metric_values[0]: .2f}s"
    b.plot()
    _ = plt.figtext(0.5, 0.8, label)